In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# GPT-2 모델과 토크나이저 로드
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 수정할 layer 번호 설정
modify_layer = 2

# Attention map을 identity matrix로 설정하는 함수
def modify_attention(module, input, output):
    if isinstance(module, torch.nn.MultiheadAttention):
        layer_num = int(module.layer_num)
        if layer_num == modify_layer:
            attention_probs = output[1]
            batch_size, num_heads, seq_length, _ = attention_probs.shape
            attention_probs.data = torch.eye(seq_length).expand(batch_size, num_heads, -1, -1)
    return output

# Hook 등록 함수
def register_hooks(model):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.MultiheadAttention):
            layer_num = name.split(".")[2]
            module.layer_num = layer_num
            module.register_forward_hook(modify_attention)

# Hook 등록
register_hooks(model)

In [7]:
# 텍스트 생성 함수
def generate_text(input_text, guidance_scale):
    # 입력 텍스트 인코딩
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)
    
    # Attention map 수정 후 forward 계산
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_state_p = outputs.hidden_states[modify_layer]
    
    # 원래 attention으로 forward 계산
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_state_o = outputs.hidden_states[modify_layer]
    
    # Hidden state 계산
    hidden_state = hidden_state_p + guidance_scale * (hidden_state_p - hidden_state_o)
    
    # 텍스트 생성
    output = model.generate(
        input_ids,
        max_length=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=attention_mask,
        logits_processor=[
            lambda input_ids, scores: scores + guidance_scale * (scores - model(inputs_embeds=hidden_state).logits[:, -1, :])
        ],
        top_p=0.9
    )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [8]:
# 텍스트 생성 및 출력
input_text = "Hello, how are you? I'm doing well."
guidance_scale = 6.0
generated_text = generate_text(input_text, guidance_scale)
print("Generated Text:")
print(generated_text)

Generated Text:
Hello, how are you? I'm doing well. Thank goodness." ―Wilson Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce Pearce


In [9]:
# Perplexity 계산 함수
def calculate_perplexity(model, tokenizer, text):
    encoded_text = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(encoded_text, labels=encoded_text)
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

In [10]:
# Perplexity 계산 및 출력
perplexity = calculate_perplexity(model, tokenizer, generated_text)
print(f"Perplexity: {perplexity:.2f}")

Perplexity: 3.57
